In [6]:
# Imports
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from src.helpers.functions import lag_matrix, select_AR_lag_SIC
from src.helpers.refactored import reduce_dimensions, loocv_ts, loocv_ts_bayes, winsor, standardize, ar_predict, scale_X
from src.helpers.autoencoder import Autoencoder
import torch
import torch.nn as nn

%reload_ext autoreload
%autoreload 2

In [7]:
def gen_data(T, N, h_steps=1, linear=True, n_fac = 3):
    """ 
    Generates data for the monte carlo simulation

    Parameters
    ----------
    T : int
        Number of time steps
    N : int
        Number of variables in X
    h_steps : int
        Number of steps ahead forecasted

    Returns
    -------
    X : ndarray
        T x N array of data
    Y : ndarray
        T x 1 array of data
    """

    # First generate the factors: f
    f = np.random.normal(size=(n_fac, T))

    # Generate the forecast target: Y
    y_h = np.sum(f, axis=0) + np.random.normal(size=(T,))
    y_t = np.roll(y_h, h_steps)

    # Generate the data: X
    X, true_f = F(f, linear=linear, N=N)

    return X, y_t, true_f

def F(f, linear=True, N=500):
    """ 
    Function to generate X from f

    Parameters
    ----------
    f : ndarray
        n_fac x T dimensional factor
    linear : bool
        Whether to generate X linearly or not

    Returns
    -------
    X : ndarray
        T x N array of data
    """
    n_fac, T = f.shape

    if linear:
        lambda_m = np.random.normal(size=(N, n_fac))

        # Generate X linearly from f by multiplying with lambda
        X = np.dot(f.T, lambda_m.T)
    else:
        # Generate X nonlinearlity by expanding f with polyonomial and sign functions
        # Then multiply with lambda

        # First generate the polyonomial(2) expansion of f
        f_poly = PolynomialFeatures(2).fit_transform(f.T)

        # Add f_poly to the factor
        f = np.concatenate((f, f_poly.T), axis=0)
        
        # Then generate the sign function expansion
        f_sign = np.sign(f)

        # Add f_sign to the factor
        f = np.concatenate((f, f_sign), axis=0)
    
        # Generate lambda for nonlinear case
        n_fac = f.shape[0]
        lambda_m = np.random.normal(size=(N, n_fac))

        # Multiply with lambda to get X
        X = np.dot(f.T, lambda_m.T)
        
    # Define the variance of the indivual variables by drawing from a uniform distribution
    variance_u = np.random.uniform(0, 1, N) 
    u = np.random.multivariate_normal(mean=np.zeros(N), cov=np.diag(variance_u), size=T)

    # Add noise
    X += u

    return X, f

In [8]:
def simulation_forecasting(X, y, T, scale_method, dim_method,nfac, hyper_params_grid, h=1):
    """
    Function to do forecasting for within the simulation study

    Parameters
    ----------

    Returns
    -------
    forecasts : ndarray
        T_test x 1 array of forecasts
    
    scales: ndarray
        T_test x N x 2 array of scaling factors

    actual_values : ndarray
        T_test x 1 array of actual values

    """ 
    T = y.shape[0]

    M = 200  # In sample periods
    N = T - M  # Out of sample periods

    forecast_spca = np.zeros((N - h, 1))  # Forecast errors of scaled PCA
    forecast_ar = np.zeros((N - h, 1))  # Forecast errors of AR model
    actual_y = np.zeros((N - h, 1))  # Actual values of y
    betas = np.zeros((N - h, X.shape[1]))  # Betas for scaling the variables
    f_hat = np.zeros((N - h, nfac))  # Estimated factors

    # Initialize the models
    ae = None
    lr = LinearRegression()

    p_max = 3 # Max number of lags for AR(p) model
    
    # Loop over all out of sample periods
    for n in range(N - h):            
        # Use all available data up to time t
        X_t = X[:(M + n), :]
        y_t = y[:M + n]

        # Get the actual value of y for the forecast period t+h
        actual_y[n] = y[M + n + h - 1]

        # Standardize the data
        X_t = standardize(X_t)

        # Get number lags
        p_AR_star_n = select_AR_lag_SIC(y_t, h, p_max=p_max)
        
        # Compute the forecast of the AR model
        forecast_ar[n] = ar_predict(y_t, p_AR_star_n, h)
        
        #### STEP 1: Scaling factors ####

        # Compute the betas for scaling the variables
        beta = scale_X(X_t, y_t, h, method=scale_method)
        
        # Winsorizing the betas
        beta_win = winsor(np.abs(beta), p=(0, 90))
        
        # Save the betas
        betas[n, :] = beta_win

        # Scale the factors by the winsorized betas
        scaleX_t = X_t * beta_win

        #### Intermezzo: Find the optimal number of factors (or other hyperparameters) ####
        
        if n == 0:
            #print("Starting hyperparameter optimization")
            if dim_method != "ae":
                hyper_params = loocv_ts(X=X_t, y=y_t, h=h, p_AR_star_n=p_AR_star_n, method=dim_method,
                                         forecast_method="ols", scale_method=scale_method, grid=hyper_params_grid)
            else:
                # Use bayesian optimization to find the optimal hyperparameters
                hyper_params = loocv_ts_bayes(X=X_t, y=y_t, h=h, p_AR_star_n=p_AR_star_n, method=dim_method,
                                               scale_method=scale_method, space=hyper_params_grid, trials=100, forecast_method="ols")
            print("-----------------------------------------------------------------")
            
            if dim_method == "ae":
                # Initialize the autoencoder
                ae = Autoencoder(input_dim=X.shape[1], activation=nn.SiLU, hyper_params=hyper_params)

                # Train the autoencoder on the in sample data
                ae.train_model(scaleX_t, lr=hyper_params.get("lr", 0.001), num_epochs=hyper_params.get("epochs", 300))
            
        #### STEP 2: Dimension Reduction ####            

        # Compute the reduced dimensionality representation of the factors
        x_spc = reduce_dimensions(X=scaleX_t, hyper_params=hyper_params, method=dim_method, dim_red_model=ae)

        #### STEP 3: Forecasting ####

        # Add lag of y_t to the factors
        if p_AR_star_n > 0:
            # Add lags of y to x
            x_spc = lag_matrix(x_spc, y_t, p_AR_star_n)

            # Remove the first p_AR_star_n observations of y_t
            y_t = y_t[(p_AR_star_n-1):]

        # Compute the forecast of the PCA and scaled PCA model
        lr.fit(x_spc[:-h], y_t[h:])
        forecast_spca[n] = lr.predict(x_spc[-1].reshape(1, -1))

    # Compute the forecast errors
    error_spca = actual_y - forecast_spca
    error_ar = actual_y - forecast_ar
    
    # Compute the R squared out of sample against the AR model
    SSE_spca = np.sum(error_spca**2)
    SSE_ar = np.sum(error_ar**2)

    print("MSE_spca: ", round(SSE_spca/len(error_ar), 10), "MSE_ar: ", round(SSE_ar/len(error_ar), 10))

    R2_spca = (1 - SSE_spca / SSE_ar)

    print("R2_spca: ", round(R2_spca * 100, 4))
    
    return forecast_spca, betas, actual_y, f_hat

In [9]:
# Defining the grids for the hyperparameters
rbf_params = {"gamma": 10**np.arange(-6,-2.5,.5)
              }

ae_params = {
    "layer_dims": [[], [16], [32], [32, 32], [64, 32], [64, 64]],
    "epochs": [200, 300],
    "update_epochs": [10],
    'batch_size': [64, 100],
    'lr': [0.001, 0.01],
    'update_lr': [0.001, 0.01],
}

parameters = {'ae': ae_params,
              'rbf': rbf_params,
              'pca': {},
}

nfac_key = {
    'ae': 'hidden_dim',
    'rbf': 'n_components',
    'pca': 'nfac',
}

In [10]:
# Run the simulation for linear and nonlinear case and save the data to file for later use

# Set the parameters
B = 100
T = 250
N = 500
h_steps = 1
linear = True
n_fac = 3
np.random.seed(42)

for linear in [True, False]:
    if linear:
        factors = n_fac
    else:
        factors = (4*n_fac + n_fac * (n_fac + 1))//2
    for scaling_method in ['none', 'regression', 'distance_correlation']:
        for dim_method in [ 'ae']:
            forecasts_full = np.zeros((B, 49)) # Forecasts
            betas_full = np.zeros((B, 49, N)) # Betas for the scaling
            actual_y_full = np.zeros((B, 49)) # Actual y values
            f_hat_full = np.zeros((B, 49, factors)) # Estimated factors

            # Set factors for the dimension method
            parameters[dim_method][nfac_key[dim_method]] = [factors]

            print('Linear: {}, scaling_method: {}, dim_method: {}'.format(linear, scaling_method, dim_method))
            for r in range(B):
                if r % 10 == 0:
                    print('r: {}'.format(r))

                # Generate the data
                X, y, f = gen_data(T=T, N=N, h_steps=1, linear=True, n_fac=n_fac)

                # Do the forecasting
                results = simulation_forecasting(X=X, y=y, T=T, h=h_steps, scale_method=scaling_method, dim_method=dim_method,
                                                    hyper_params_grid=parameters[dim_method], nfac=factors)

                # Extract the results
                forecasts, betas, actual_values, f_hat = results

                # Save the results in the full arrays
                forecasts_full[r] = forecasts.flatten()
                betas_full[r] = betas
                actual_y_full[r] = actual_values.flatten()
                f_hat_full[r] = f_hat
                
            # Save the results to file
            np.save('resources/results/monte_carlo/forecasts_{}_{}_{}_{}.npy'.format(linear, scaling_method, dim_method, h_steps), forecasts_full)
            np.save('resources/results/monte_carlo/scales_{}_{}_{}_{}.npy'.format(linear, scaling_method, dim_method, h_steps), betas_full)
            np.save('resources/results/monte_carlo/actual_values_{}_{}_{}_{}.npy'.format(linear, scaling_method, dim_method, h_steps), actual_y_full)   
            np.save('resources/results/monte_carlo/f_hat_{}_{}_{}_{}.npy'.format(linear, scaling_method, dim_method, h_steps), f_hat_full)

Linear: True, scaling_method: none, dim_method: ae
r: 0
100%|██████████| 100/100 [08:59<00:00,  5.39s/trial, best loss: 1.025818515672226]
{'batch_size': 0, 'dropout': 0.14031156906676245, 'epochs': 1, 'gauss_noise': 0.5878813698701918, 'hidden_dim': 0, 'layer_dims': 5, 'lr': 0, 'update_epochs': 0, 'update_lr': 1}
Best model configuration:  {'batch_size': 64, 'dropout': 0.14031156906676245, 'epochs': 300, 'gauss_noise': 0.5878813698701918, 'hidden_dim': 3, 'layer_dims': [64, 64], 'lr': 0.001, 'update_epochs': 10, 'update_lr': 0.01}
-----------------------------------------------------------------
MSE_spca:  1.0674861438 MSE_ar:  4.4353635282
R2_spca:  75.9324
100%|██████████| 100/100 [07:35<00:00,  4.55s/trial, best loss: 0.9863935667844541]
{'batch_size': 0, 'dropout': 0.07786588358835148, 'epochs': 1, 'gauss_noise': 0.3261964737782854, 'hidden_dim': 0, 'layer_dims': 2, 'lr': 0, 'update_epochs': 0, 'update_lr': 0}
Best model configuration:  {'batch_size': 64, 'dropout': 0.077865883588